# Исследование надежности заемщиков


Во второй части проекта вы выполните шаги 3 и 4. Их вручную проверит ревьюер.
Чтобы вам не пришлось писать код заново для шагов 1 и 2, мы добавили авторские решения в ячейки с кодом. 



## Откройте таблицу и изучите общую информацию о данных

In [2]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [3]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

In [5]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [6]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

In [7]:
data['days_employed'] = data['days_employed'].abs()

In [8]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть. Тем более этот столбец не понадобится вам для исследования.

In [9]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [10]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [11]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

In [12]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [13]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

In [14]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

In [15]:
data['education'] = data['education'].str.lower()

In [16]:
data.duplicated().sum()

71

In [17]:
data = data.drop_duplicates()

### Категоризация данных

In [18]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [19]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [20]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [21]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [22]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Шаг 3. Исследуйте данные и ответьте на вопросы

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

Сгруппируем датасет по количеству детей и проблем с долгами, а также посчитаем количество значений для дальнейшего проведения анализа

In [23]:
child_grouped = data.groupby('children')
child_grouped_count = child_grouped['debt'].value_counts()
child_grouped_count # Запишем результат value_counts() в отдельную переменную для того, чтобы обращаться к ней в дальнейшем

children  debt
0         0       13028
          1        1063
1         0        4364
          1         444
2         0        1858
          1         194
3         0         303
          1          27
4         0          37
          1           4
5         0           9
Name: debt, dtype: int64

Посчитаем конверсию проблемных клиентов банка в зависимости от количества детей в их семьях, так как в семьях с пятью детьми не в датасете не найдено проблемных клиентов примем их конверсвию за 0.0 и добавим в полученный список после цикла

In [24]:
'''
Импортируем библиотеку numpy для возможности поделить элементы одного списка на элементы другого, это необходимо для того чтобы
расчитать конверсию и ввести ее в список
'''
import numpy as np

total_chield_list = [] # Пустой список для количества заемщиков

for i in range(2, 14, 2): # Через срез value_counts() обращаемся к каждой категори, а затем суммируем два значения
    total_num = child_grouped_count[i-2:i].sum() 
    total_chield_list.append(total_num)
    
total_chield_debt = [] # Пустой список для количества должников

for i in range(5): 
    total_num = child_grouped_count[i][1]
    total_chield_debt.append(total_num)

total_chield_debt.append(0)# Добавим в конец количество должников для категории "5 детей"

# Создаем numpy-списки для расчета конверсии категорий
total_chield_numpy = np.array(total_chield_list)
total_chield_debt_numpy = np.array(total_chield_debt)

conversions_num_chield = total_chield_debt / total_chield_numpy # Конверсии в виде numpy-списков

In [25]:
'''
Создаем и выводим датафрейм со сводными данными
'''
chield_pivot = pd.DataFrame({'Количество детей': range(6), 'Количество заемщиков': total_chield_list,
        'Количество должников': total_chield_debt, 'Процент просрочки (%)': conversions_num_chield*100})

chield_pivot

,Количество детей,Количество заемщиков,Количество должников,Процент просрочки (%)
0,0,14091,1063,7.543822
1,1,4808,444,9.234609
2,2,2052,194,9.454191
3,3,330,27,8.181818
4,4,41,4,9.756098
5,5,9,0,0.000000


In [29]:
x = data.pivot_table(index='children', values='debt', aggfunc=['count', 'sum', 'mean'])
x.columns = ['debt', 'info', 'debt']
x

,debt,info,debt
children,,,
0,14091,1063,0.075438
1,4808,444,0.092346
2,2052,194,0.094542
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000


Для наглядности полученных результатов построим простую гистаграмму:

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams["axes.grid"] = True # Переопределим параметр библиотеки для отображения сетки на графике
index = range(6) # Определим координаты, пусть для значений оси Х будет сгенерирована последовательность чисел от 0 до 5
values = conversions_num_chield
plt.title('Конверсия по количеству детей')
plt.xlabel('Количество детей')
plt.bar(index,values) # Построим гистаграмму в вертикально положении
plt.show()

**Вывод:** Из полученных данных можно подумать, что в семьях в которых 5 детей, нет проблем с задолженностью, но нет, так как таких клиентов незначительое количество по сравнению с общим числом, пренебрегаем этой группой, для того чтобы провести более подробный анализ конверсии для семей с пятью детьми необходимо иметь большее количество данных. Однако, имеющиеся данные позволяют сказать, что существует выраженная зависимость между количеством детей и возвратом кредита в срок, конверсия для бездетных семей показывает более лучшие результаты по сравнению с другими около 8 процентов проблемных и далее резко растет пытаясь соблюсти пропорцию: чем больше детей, тем выше конверсия, поэтому можно сделать вывод о том, что чем больше детей у клиента тем выше вероятность кредитных проблем, в данном датафрейме можно сделать интересное наблюдение: в семьях с тремя детьми наблюдается довольно резкое улучшение конверсии она приближиается к уровню бездетных семей и имеет конверсию 8.9%, в то время как бездетные семьи имеют около 8% конверсии

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

Вначале, сгруппируем данные по семейному положению и посчитаем количество для каждой категории семейного положения, разделим категорию на тех кто имел проблемы с долгом и тех кто не имел проблем. 
    Обратим внимание, что к значениям value_counts() можно обращаться по индексу по два значения для каждой категории всего категорий пять и проведем анализ категориальных значений

In [ ]:
# Ваш код будет здесь. Вы можете создавать новые ячейки.
family_data_group = data.groupby('family_status')
family_data_count = family_data_group['debt'].value_counts()
family_data_count # Запишем результат value_counts() в отдельную переменную для того, чтобы обращаться к ней в дальнейшем

Далее, посчитаем конверсию для семейных положений через цикл for с диапазоном значений от 2 до 10 с шагом 2 для правильности расчета и добавим каждое значение в список:

In [ ]:
family_status_total = [] # Пустой список для общего числа в категории "Семейное положение"

for i in range(2, 12, 2):
    total_family_num = family_data_count[i-2] + family_data_count[i-1]
    family_status_total.append(total_family_num)
    
family_status_total_debt = [] # Пустой список для количества должников в категории "Семейное положение"

for i in range(1, 11, 2):
    family_status_total_debt.append(family_data_count[i])

conversions_family_status = [] # Пустой список для конверсий в категории "Семейное положение"

# Создаем numpy-списки для расчета конверсии категорий
family_status_total_numpy = np.array(family_status_total)
family_total_debt_numpy = np.array(family_status_total_debt)

conversions_family_status = family_total_debt_numpy / family_status_total_numpy # Конверсии в виде numpy-списков

Используем цикл for для перебора значений по диапазону уникальных имен в сгруппированном фрейме данных для правильности названий значений конверсий на гистаграмме:

In [ ]:
name_lines_family = []

# Переберем с помощью цикла for уникальные значения имен и добавим их в пустой список
for i in range(5):
    name_lines_family.append(family_data_group['family_status'].unique()[i][0])
name_lines_family

In [ ]:
'''
Создаем и выводим датафрейм со сводными данными
'''
family_pivot = pd.DataFrame({'Семейное положение': name_lines_family, 'Количество заемщиков': family_status_total,
        'Количество должников': family_status_total_debt, 'Процент просрочки (%)': conversions_family_status*100})

family_pivot

Для наглядности полученных результатов построим простую гистаграмму:

In [ ]:
index = name_lines_family # Определим координаты
values = conversions_family_status
plt.title('Конверсия по семейным категориям')
plt.barh(index,values) # Построим гистаграмму в горизонтальном положении
plt.show()

**Вывод:** Из полученных результатов можно сделать вывод о том, что овдовевшие люди меньше других подверженны проблемам с долгами, однако, нельзя не учесть тот факт, количество женатых и замужних людей кратно превосходит любую другую семейную категорию, но исходя из имеющегося фрейма данных можно наблюдать зависимость семейного положения на возврат кредита в срок, овдовевшие и находящиеся в разводе люди меньше прочих подверженны "проблемности" по кредитным долгам около 7 процентов против не женатых, не замужних людей с конверсией почти в 11 процентов - максимальное значение конверсии среди всех категорий

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Аналогично предыдущим пунктам также вычислим конверсии для разных категорий, использовав группировку и подсчеты количества значений с помощью метода value_counts() и обращаться к сумме подсчета будем также по индексу. Первое что сделаем - сгруппируем фрейм данных по новому столбцу, что был создан в результате работы функции categorize_income() категорезирующая клиентов по ежемесячным доходам и проведем анализ категориальных значений

In [ ]:
# Ваш код будет здесь. Вы можете создавать новые ячейки.
total_income_grouped = data.groupby('total_income_category')
total_income_count = total_income_grouped['debt'].value_counts()
total_income_count

Видим, что самая многочисленная категория "С" кратно превосходит прочие категории, но делать какие-либо вывода еще рано. Расчитаем сначала конверсии проблемных клиентов этих категорий

In [ ]:
total_income_num = [] # Пустой список для общего числа в категории "Уровень дохода"

for i in range(2, 12, 2):
    total_income = total_income_count[i-2] + total_income_count[i-1]
    total_income_num.append(total_income)
    
family_income_debt = [] # Пустой список для количества должников в категории "Уровень дохода"

for i in range(1, 11, 2):
    family_income_debt.append(total_income_count[i])
    
total_income_numpy = np.array(total_income_num)  
total_income_debt_numpy = np.array(family_income_debt)

conversions_total_income = total_income_debt_numpy / total_income_numpy # Расчет конверсий для категорий дохода

Используем цикл for для перебора значений по диапазону уникальных имен в сгруппированном фрейме данных для правильности названий значений конверсий на гистаграмме:

In [ ]:
name_lines_income = []

# Переберем с помощью цикла for уникальные значения имен и добавим их в пустой список
for i in range(5):
    name_lines_income.append(total_income_grouped['total_income_category'].unique()[i][0])
name_lines_income

In [ ]:
income_pivot = pd.DataFrame({'Категория дохода': name_lines_income, 'Количество заемщиков': total_income_num,
        'Количество должников': family_income_debt, 'Процент просрочки (%)': conversions_total_income*100})

income_pivot

Для наглядности полученных результатов построим простую гистаграмму:

In [ ]:
index = name_lines_income # Определим координаты
values = conversions_total_income
plt.title('Конверсия по уровню дохода')
plt.barh(index,values) # Построим гистаграмму в горизонтальном положении
plt.show()

**Вывод:** На графике наглядно видна зависимость между уровнем дохода и возвратом кредита в срок, так клиенты категории "D" клиенты с доходами 30001–50000 руб по численности на третьем месте во всем датафрейме самые непроблемные имеющие самую низкую конверсию около 6.4%, в то время самыми проблемными клиентами являются люди категории "E" с доходами от 0 до 30000 руб. Однако, эта зависимость не пропорциональна доходам, так клиенты со средними доходами и имеющие самую лучшую конверсию легко могут перейти в самую "тяжелую" группу так как находятся от них недалеко, а самая многчисленная группа - "C" на уровень выше имеет уже конверсию немногим меньше самой проблемной категории на уровне 9.3%. Таким образом, лучшая категория имеет все шансы на переход к более худшей конверсии, если переместится вверх, либо вниз по категории доходов.

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

Аналогично предыдущему пункту также вычислим конверсии для разных категорий, использовав группировку и подсчеты количества значений с помощью метода value_counts() и обращаться к сумме подсчета будем также по индексу. Первое что сделаем - сгруппируем фрейм данных по новому столбцу, что был создан в результате работы функции categorize_purpose() категорезирующая клиентов по целям кредита и проведем анализ категориальных значений

In [ ]:
# Ваш код будет здесь. Вы можете создавать новые ячейки.
total_purpose_grouped = data.groupby('purpose_category')
total_purpose_count = total_purpose_grouped['debt'].value_counts()
total_purpose_count # Запишем результат value_counts() в отдельную переменную для того, чтобы обращаться к ней в дальнейшем

In [ ]:
total_purpose_category = [] # Пустой список для общего количества в категории

for index_count in range(2, 10, 2):
    total = total_purpose_count[index_count-1] + total_purpose_count[index_count-2]
    total_purpose_category.append(total)
    
total_purpose_debt = [] # Пустой список для количества должников

for i in range(1, 9, 2):
    total_purpose_debt.append(total_purpose_count[i])

total_purpose_category_numpy = np.array(total_purpose_category)
total_purpose_debt_numpy = np.array(total_purpose_debt)
conversions_purpose_category = total_purpose_debt_numpy / total_purpose_category_numpy # numpy-списки конверсий целей кредита

Используем цикл for для перебора значений по диапазону уникальных имен в сгруппированном фрейме данных для правильности названий значений конверсий на гистаграмме:

In [ ]:
name_lines_purpose = []

# Переберем с помощью цикла for уникальные значения имен и добавим их в пустой список
for i in range(4):
    name_lines_purpose.append(total_purpose_grouped['purpose_category'].unique()[i][0])
name_lines_purpose

In [ ]:
purpose_pivot = pd.DataFrame({'Цели кредита': name_lines_purpose, 'Количество заемщиков': total_purpose_category,
        'Количество должников': total_purpose_debt, 'Процент просрочки (%)': conversions_purpose_category*100})

purpose_pivot

Для наглядности полученных результатов построим простую гистаграмму:

In [ ]:
index = name_lines_purpose # Определим координаты
values = conversions_purpose_category
plt.title('Конверсия по целям кредита')
plt.barh(index,values) # Построим гистаграмму в горизонтальном положении
plt.show()

**Вывод:** Из полученных результатов видно, что на операции с недвижимостью приходится самая низкая она же лучшая конверсия для банков, отвечая на поставленый вопрос можно сказать, что разные цели кредита влияют на его возврат в срок, так для текущего датафрейма операции с недвижимостью конверсия составляет чуть менее 8%, для проведения свадьбы - 8.6%, для получения образования - на уровне 10%, для операций с автомобилем - 10.3% - самое высокое значение. Самое многочисленное количество приходится как раз на самую низку конверсию - на операции с недвижимостью, самое малочисленное количество - на проведение свадьб.

#### 3.5 Приведите возможные причины появления пропусков в исходных данных.

*Ответ:* Ошибки при запросах к базам данных, ошибки записи в базах данных, а также грамотность кода, которая должна минимизировать человеческий фактор при записи, например, возможность пропуска обязательных для заполнения полей. Возможные техсбои которые могут не сохранить часть данных в базе данных

#### 3.6 Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

*Ответ:* Медианна берет самое среднее значение не позволяя перетягивать в какую-либо сторону значение как это может быть при расчете среднего числа из прочих данных, например, из-за аномальных данных, так в коде метод сортирует данные и если они нечетного количества то медианна - среднее значение, если четное, то берет два значения стоящие в центре данных и высчитывает между ними среднее, это и есть медианна. Отвечая на поставленный вопрос, можно сказать, что медианна в условиях отсутствующих количественных значений и имеющих большую выборку данных, подставить в пропуски медианну значений по категориям - лучшее решение из-за алгоритма, описанного выше.

### Шаг 4: общий вывод.

Проведя поверхностный анализ и отвечая на поставленные вопросы, можно описать наиболее "идеального" клиента для банка: это бездетный человек имеющий доход 30-50 тыс. руб, желательно вдовец или вдова или находящееся в разводе и имеющий цель кредита на операции с недвижимостью. Конечно, можно задать больше вопросов, как зависит возраст на выплату кредита в срок или пол, а также исследовать количество целевой категории для ее сравнения с другими категориями, чтобы получить преположительно наиболее верные зависимости, если учитывать их долю в общем числе, но все это не отвечает напрямую на поставленные вопросы лишь возможно скорректируют их, в этом можно "зарыться", даже самые скудные данные можно анализировать продолжительное время. Разумеется, для более глубокого анализа нельзя не учитывать, например, долю категории в общем фрейме данных.

В датасете были обработали дубликаты, пропуски, в числовых значениях, например, таких как уровень ежемесячного дохода вместо пропусков вставили медианные значения по категории, категоризацю данных провели в столбцах по уровню дохода и цели получения кредита. Как было сказано выше, в некоторых категориях было явно недостаточно записей для наиболее качественного анализа, так например, в семьях с 5 детьми количество должников равняется нулю, а заемщиков в этой категории - 9, при общем объеме более чем в 21 тыс. Если есть возможность, рекомендуется пополнение датасета малочисленными категориями в целях более качественного анализа

Интересное наблюдение: в семьях с тремя детьми конверсия должников резко падает, затем с четвертым ребенком возвращается в восходящий тренд, также при операциях с недвижимостью конверсия смотрится гораздо лучше, чем при других категориях целей кредита, а люди, зарабатывающие 30-50 тыс руб более всего склонны возвращать кредиты без просрочек, однако, с увеличением или понижением категории дохода (ниже 30 тыс или выше 50 тыс руб) конверсия резко ухудшается